In [ ]:
# 구글 드라이브 연동
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [ ]:
#dir_ = 'C:/Users/yeonjun.in/Desktop/SUBMISSION MODEL/' # input only here
dir_ = '/content/gdrive/My Drive/data/'

#### setting other directory

In [ ]:
raw_data_dir = dir_
processed_data_dir = dir_+'processed_data/'
log_dir = dir_+'log/'
model_dir = dir_+'model/'

In [ ]:
####################################################################################
##################### 1-3. recursive model by store & dept #########################
####################################################################################

In [ ]:
ver, KKK = 'priv', 0

In [ ]:
STORES = ['CA_1', 'CA_2', 'CA_3', 'CA_4', 'TX_1', 'TX_2', 'TX_3', 'WI_1', 'WI_2', 'WI_3']
DEPTS = ['HOBBIES_1', 'HOBBIES_2', 'HOUSEHOLD_1', 'HOUSEHOLD_2', 'FOODS_1', 'FOODS_2', 'FOODS_3']

In [ ]:
import numpy as np
import pandas as pd
import os, sys, gc, time, warnings, pickle, psutil, random

from multiprocessing import Pool

warnings.filterwarnings('ignore')

In [ ]:
########################### Helpers
#################################################################################
## Seeder
def seed_everything(seed=0):
    random.seed(seed)
    np.random.seed(seed)

    
## Multiprocess Runs
def df_parallelize_run(func, t_split):
    num_cores = np.min([N_CORES,len(t_split)])
    pool = Pool(num_cores)
    df = pd.concat(pool.map(func, t_split), axis=1)
    pool.close()
    pool.join()
    return df

In [ ]:
########################### Helper to load data by store ID
#################################################################################
# Read data
def get_data_by_store(store, dept):
    
    # Read and contact basic feature
    df = pd.concat([pd.read_pickle(BASE),
                    pd.read_pickle(PRICE).iloc[:,2:],
                    pd.read_pickle(CALENDAR).iloc[:,2:]],
                    axis=1)
    
    df = df[df['d']>=START_TRAIN]
    
    df = df[(df['store_id']==store) & (df['dept_id']==dept)]

    df2 = pd.read_pickle(MEAN_ENC)[mean_features]
    df2 = df2[df2.index.isin(df.index)]
        
    df3 = pd.read_pickle(LAGS).iloc[:,3:]
    df3 = df3[df3.index.isin(df.index)]
    
    df = pd.concat([df, df2], axis=1)
    del df2
    
    df = pd.concat([df, df3], axis=1)
    del df3
    
    features = [col for col in list(df) if col not in remove_features]
    df = df[['id','d',TARGET]+features]
    
    df = df.reset_index(drop=True)
    
    return df, features

# Recombine Test set after training
def get_base_test():
    base_test = pd.DataFrame()

    for store_id in STORES:
        for state_id in DEPTS:
            temp_df = pd.read_pickle(processed_data_dir+'test_'+store_id+'_'+state_id+'.pkl')
            temp_df['store_id'] = store_id
            temp_df['dept_id'] = state_id
            base_test = pd.concat([base_test, temp_df]).reset_index(drop=True)
    
    return base_test


########################### Helper to make dynamic rolling lags
#################################################################################
def make_lag(LAG_DAY):
    lag_df = base_test[['id','d',TARGET]]
    col_name = 'sales_lag_'+str(LAG_DAY)
    lag_df[col_name] = lag_df.groupby(['id'])[TARGET].transform(lambda x: x.shift(LAG_DAY)).astype(np.float16)
    return lag_df[[col_name]]


def make_lag_roll(LAG_DAY):
    shift_day = LAG_DAY[0]
    roll_wind = LAG_DAY[1]
    lag_df = base_test[['id','d',TARGET]]
    col_name = 'rolling_mean_tmp_'+str(shift_day)+'_'+str(roll_wind)
    lag_df[col_name] = lag_df.groupby(['id'])[TARGET].transform(lambda x: x.shift(shift_day).rolling(roll_wind).mean())
    return lag_df[[col_name]]

In [ ]:
########################### Model params
#################################################################################
import lightgbm as lgb
lgb_params = {
                    'boosting_type': 'gbdt',
                    'objective': 'tweedie',
                    'tweedie_variance_power': 1.1,
                    'metric': 'rmse',
                    'subsample': 0.5,
                    'subsample_freq': 1,
                    'learning_rate': 0.015,
                    'num_leaves': 2**8-1,
                    'min_data_in_leaf': 2**8-1,
                    'feature_fraction': 0.5,
                    'max_bin': 100,
                    'n_estimators': 3000,
                    'boost_from_average': False,
                    'verbose': -1
                } 

In [ ]:
########################### Vars
#################################################################################
VER = 1                          
SEED = 42                        
seed_everything(SEED)            
lgb_params['seed'] = SEED        
N_CORES = psutil.cpu_count()     


#LIMITS and const
TARGET      = 'sales'            
START_TRAIN = 700                
END_TRAIN   = 1941 - 28*KKK      
P_HORIZON   = 28                 
USE_AUX     = False             

remove_features = ['id','cat_id', 'state_id','store_id','dept_id',
                   'date','wm_yr_wk','d',TARGET]
mean_features   = ['enc_item_id_store_id_mean','enc_item_id_store_id_std'] 
#mean_features   = ['enc_item_id_cat_id_mean','enc_item_id_cat_id_std','enc_item_id_dept_id_mean','enc_item_id_dept_id_std'] 

ORIGINAL = raw_data_dir
BASE     = processed_data_dir+'grid_part_1.pkl'
PRICE    = processed_data_dir+'grid_part_2.pkl'
CALENDAR = processed_data_dir+'grid_part_3.pkl'
LAGS     = processed_data_dir+'lags_df_28.pkl'
MEAN_ENC = processed_data_dir+'mean_encoding_df.pkl'
#MEAN_ENC = processed_data_dir+'mean_encoding_df_test.pkl'


#SPLITS for lags creation
SHIFT_DAY  = 28
N_LAGS     = 15
LAGS_SPLIT = [col for col in range(SHIFT_DAY,SHIFT_DAY+N_LAGS)]
ROLS_SPLIT = []
for i in [1,7,14]:
    for j in [7,14,30,60]:
        ROLS_SPLIT.append([i,j])

In [ ]:
########################### Train Models
#################################################################################
for store_id in STORES:
    for state_id in DEPTS:
        print('Train', store_id, state_id)

        grid_df, features_columns = get_data_by_store(store_id, state_id)

        train_mask = grid_df['d']<=END_TRAIN
        valid_mask = train_mask&(grid_df['d']>(END_TRAIN-P_HORIZON))
        preds_mask = (grid_df['d']>(END_TRAIN-100)) & (grid_df['d'] <= END_TRAIN+P_HORIZON)

        train_data = lgb.Dataset(grid_df[train_mask][features_columns], 
                           label=grid_df[train_mask][TARGET])

        valid_data = lgb.Dataset(grid_df[valid_mask][features_columns], 
                           label=grid_df[valid_mask][TARGET])

        grid_df = grid_df[preds_mask].reset_index(drop=True)
        keep_cols = [col for col in list(grid_df) if '_tmp_' not in col]
        grid_df = grid_df[keep_cols]

        d_sales = grid_df[['d','sales']]
        substitute = d_sales['sales'].values
        substitute[(d_sales['d'] > END_TRAIN)] = np.nan
        grid_df['sales'] = substitute

        #grid_df.to_pickle(processed_data_dir+'test_'+store_id+'_'+state_id+'.pkl')
        grid_df.to_pickle(processed_data_dir+'test_'+store_id+'_'+state_id+'_test'+'.pkl')
        del grid_df, d_sales, substitute

        seed_everything(SEED)
        estimator = lgb.train(lgb_params,
                              train_data,
                              valid_sets = [valid_data],
                              verbose_eval = 100
                              
                              )
        
        display(pd.DataFrame({'name':estimator.feature_name(),
                              'imp':estimator.feature_importance()}).sort_values('imp',ascending=False).head(25))
        
        #model_name = model_dir+'lgb_model_'+store_id+'_'+state_id+'_v'+str(VER)+'.bin'
        model_name = model_dir+'lgb_model_'+store_id+'_'+state_id+'_v'+str(VER)+'_test'+'.bin'
        pickle.dump(estimator, open(model_name, 'wb'))

        del train_data, valid_data, estimator
        gc.collect()

        MODEL_FEATURES = features_columns

Train CA_1 HOBBIES_1
[100]	valid_0's rmse: 2.47672
[200]	valid_0's rmse: 2.3865
[300]	valid_0's rmse: 2.3458
[400]	valid_0's rmse: 2.30804
[500]	valid_0's rmse: 2.27167
[600]	valid_0's rmse: 2.23792
[700]	valid_0's rmse: 2.20448
[800]	valid_0's rmse: 2.1695
[900]	valid_0's rmse: 2.13561
[1000]	valid_0's rmse: 2.10315
[1100]	valid_0's rmse: 2.07148
[1200]	valid_0's rmse: 2.04244
[1300]	valid_0's rmse: 2.01076
[1400]	valid_0's rmse: 1.98043
[1500]	valid_0's rmse: 1.95448
[1600]	valid_0's rmse: 1.92544
[1700]	valid_0's rmse: 1.8968
[1800]	valid_0's rmse: 1.86878
[1900]	valid_0's rmse: 1.84253
[2000]	valid_0's rmse: 1.81568
[2100]	valid_0's rmse: 1.79048
[2200]	valid_0's rmse: 1.76718
[2300]	valid_0's rmse: 1.74175
[2400]	valid_0's rmse: 1.71696
[2500]	valid_0's rmse: 1.69518
[2600]	valid_0's rmse: 1.6722
[2700]	valid_0's rmse: 1.64995
[2800]	valid_0's rmse: 1.62738
[2900]	valid_0's rmse: 1.60617
[3000]	valid_0's rmse: 1.58465


,name,imp
0,item_id,105254
20,tm_d,23741
21,tm_w,23164
55,rolling_std_180,19644
54,rolling_mean_180,18994
56,rolling_mean_tmp_1_7,18543
47,rolling_std_7,18153
53,rolling_std_60,17620
49,rolling_std_14,17526
13,event_name_1,17068


Train CA_1 HOBBIES_2
[100]	valid_0's rmse: 0.771516
[200]	valid_0's rmse: 0.75456
[300]	valid_0's rmse: 0.745319
[400]	valid_0's rmse: 0.732796
[500]	valid_0's rmse: 0.720472
[600]	valid_0's rmse: 0.705926
[700]	valid_0's rmse: 0.691826
[800]	valid_0's rmse: 0.676315
[900]	valid_0's rmse: 0.663192
[1000]	valid_0's rmse: 0.649374
[1100]	valid_0's rmse: 0.635778
[1200]	valid_0's rmse: 0.621368
[1300]	valid_0's rmse: 0.607865
[1400]	valid_0's rmse: 0.593526
[1500]	valid_0's rmse: 0.580459
[1600]	valid_0's rmse: 0.566983
[1700]	valid_0's rmse: 0.554141
[1800]	valid_0's rmse: 0.542339
[1900]	valid_0's rmse: 0.53161
[2000]	valid_0's rmse: 0.519192
[2100]	valid_0's rmse: 0.50902
[2200]	valid_0's rmse: 0.498687
[2300]	valid_0's rmse: 0.489682
[2400]	valid_0's rmse: 0.479959
[2500]	valid_0's rmse: 0.470388
[2600]	valid_0's rmse: 0.462975
[2700]	valid_0's rmse: 0.455262
[2800]	valid_0's rmse: 0.447795
[2900]	valid_0's rmse: 0.441492
[3000]	valid_0's rmse: 0.434601


,name,imp
0,item_id,77677
55,rolling_std_180,36377
20,tm_d,36264
21,tm_w,33593
54,rolling_mean_180,33461
53,rolling_std_60,29786
51,rolling_std_30,23396
25,tm_dw,20563
52,rolling_mean_60,20448
67,rolling_mean_tmp_14_60,19877


Train CA_1 HOUSEHOLD_1
[100]	valid_0's rmse: 1.58106
[200]	valid_0's rmse: 1.50097
[300]	valid_0's rmse: 1.48253
[400]	valid_0's rmse: 1.46859
[500]	valid_0's rmse: 1.4554
[600]	valid_0's rmse: 1.44346
[700]	valid_0's rmse: 1.43189
[800]	valid_0's rmse: 1.42147
[900]	valid_0's rmse: 1.41201
[1000]	valid_0's rmse: 1.40229
[1100]	valid_0's rmse: 1.39264
[1200]	valid_0's rmse: 1.38315
[1300]	valid_0's rmse: 1.37412
[1400]	valid_0's rmse: 1.366
[1500]	valid_0's rmse: 1.35734
[1600]	valid_0's rmse: 1.3486
[1700]	valid_0's rmse: 1.34055
[1800]	valid_0's rmse: 1.3324
[1900]	valid_0's rmse: 1.3249
[2000]	valid_0's rmse: 1.31701
[2100]	valid_0's rmse: 1.30945
[2200]	valid_0's rmse: 1.30122
[2300]	valid_0's rmse: 1.29306
[2400]	valid_0's rmse: 1.28586
[2500]	valid_0's rmse: 1.2794
[2600]	valid_0's rmse: 1.27207
[2700]	valid_0's rmse: 1.26546
[2800]	valid_0's rmse: 1.25935
[2900]	valid_0's rmse: 1.25282
[3000]	valid_0's rmse: 1.24686


...

,name,imp
0,item_id,96692
20,tm_d,28632
21,tm_w,25763
55,rolling_std_180,23043
54,rolling_mean_180,22728
53,rolling_std_60,19817
51,rolling_std_30,19193
49,rolling_std_14,18805
11,price_momentum_m,17661
52,rolling_mean_60,17654


Train WI_3 FOODS_2
[100]	valid_0's rmse: 1.79681
[200]	valid_0's rmse: 1.66943
[300]	valid_0's rmse: 1.63742
[400]	valid_0's rmse: 1.61586
[500]	valid_0's rmse: 1.59393
[600]	valid_0's rmse: 1.57463
[700]	valid_0's rmse: 1.55866
[800]	valid_0's rmse: 1.54331
[900]	valid_0's rmse: 1.52693
[1000]	valid_0's rmse: 1.51112
[1100]	valid_0's rmse: 1.49633
[1200]	valid_0's rmse: 1.48421
[1300]	valid_0's rmse: 1.46954
[1400]	valid_0's rmse: 1.45678
[1500]	valid_0's rmse: 1.44273
[1600]	valid_0's rmse: 1.43023
[1700]	valid_0's rmse: 1.41754
[1800]	valid_0's rmse: 1.40528
[1900]	valid_0's rmse: 1.39354
[2000]	valid_0's rmse: 1.38064
[2100]	valid_0's rmse: 1.36985
[2200]	valid_0's rmse: 1.35801
[2300]	valid_0's rmse: 1.34714
[2400]	valid_0's rmse: 1.33528
[2500]	valid_0's rmse: 1.32509
[2600]	valid_0's rmse: 1.31638
[2700]	valid_0's rmse: 1.30468
[2800]	valid_0's rmse: 1.29361
[2900]	valid_0's rmse: 1.28328
[3000]	valid_0's rmse: 1.27417


,name,imp
0,item_id,117808
20,tm_d,24952
21,tm_w,23346
54,rolling_mean_180,21294
55,rolling_std_180,20568
11,price_momentum_m,17988
53,rolling_std_60,17615
49,rolling_std_14,17011
56,rolling_mean_tmp_1_7,16803
57,rolling_mean_tmp_1_14,16657


Train WI_3 FOODS_3
[100]	valid_0's rmse: 3.32451
[200]	valid_0's rmse: 2.7707
[300]	valid_0's rmse: 2.6815
[400]	valid_0's rmse: 2.63372
[500]	valid_0's rmse: 2.59731
[600]	valid_0's rmse: 2.56766
[700]	valid_0's rmse: 2.53704
[800]	valid_0's rmse: 2.51473
[900]	valid_0's rmse: 2.48666
[1000]	valid_0's rmse: 2.45916
[1100]	valid_0's rmse: 2.43745
[1200]	valid_0's rmse: 2.42046
[1300]	valid_0's rmse: 2.40101
[1400]	valid_0's rmse: 2.38292
[1500]	valid_0's rmse: 2.36156
[1600]	valid_0's rmse: 2.34398
[1700]	valid_0's rmse: 2.32676
[1800]	valid_0's rmse: 2.31026
[1900]	valid_0's rmse: 2.29346
[2000]	valid_0's rmse: 2.2782
[2100]	valid_0's rmse: 2.26217
[2200]	valid_0's rmse: 2.24667
[2300]	valid_0's rmse: 2.23361
[2400]	valid_0's rmse: 2.21947
[2500]	valid_0's rmse: 2.20515
[2600]	valid_0's rmse: 2.19354
[2700]	valid_0's rmse: 2.18048
[2800]	valid_0's rmse: 2.16598
[2900]	valid_0's rmse: 2.15279
[3000]	valid_0's rmse: 2.13966


,name,imp
0,item_id,133932
13,event_name_1,25735
20,tm_d,24338
21,tm_w,22832
56,rolling_mean_tmp_1_7,19276
55,rolling_std_180,16577
54,rolling_mean_180,16124
57,rolling_mean_tmp_1_14,15919
47,rolling_std_7,15021
25,tm_dw,14791
